In [1]:
from __future__ import print_function
import glob
from itertools import chain
import os
import cv2
import csv
import random
import zipfile
import os.path as osp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from functools import reduce
import torch.nn as nn
from einops import rearrange, repeat
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import pandas as pd
from torchvision.transforms import ToPILImage
# from linformer import Linformer
from PIL import Image
from imgaug import augmenters as iaa
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
# from vit_pytorch.efficient import ViT
# from model import BiSeNet
import torchvision.transforms as transforms
from skimage import io, img_as_float
import timm
from sklearn.metrics import log_loss, f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score


In [2]:
seed = 17
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [3]:
device = 'cuda'
# device = 'cpu'

In [4]:
fakeavceleb_paths_df = pd.read_csv('File_Path_Dictionaries/fakeavceleb_paths.csv')
celebdf_paths_df = pd.read_csv('File_Path_Dictionaries/celebdf_paths.csv')
dfdc_paths_df = pd.read_csv('File_Path_Dictionaries/dfdc_paths.csv')
faceswap_paths_df = pd.read_csv('File_Path_Dictionaries/ff_paths.csv')


In [5]:
test_set_paths_dictionary = {
                            'FakeAVCeleb': [fakeavceleb_paths_df['Path'], fakeavceleb_paths_df['Label']], 
                            'DFDC': [dfdc_paths_df['Path'], dfdc_paths_df['Label']],
                            'CelebDFV2': [celebdf_paths_df['Path'], celebdf_paths_df['Label']],
                            'FaceForensics': [faceswap_paths_df['Path'], faceswap_paths_df['Label']]
                            }

In [6]:
import timm
models = ['xception', 'res2net101_26w_4s', 'tf_efficientnet_b7_ns', 'vit_base_patch16_224', 'swin_base_patch4_window7_224', 'mvitv2_base']
augs = ['No_Augmentations', 'Random_Cut_Out_Augs', 'Face_Cut_Out_Augs']
datasets = ['FakeAVCeleb', 'FaceForensics', 'DFDC', 'CelebDFV2']


In [7]:
def get_scores(predictions):
    true_labels = []
    predicted_labels = []
    predicted_probs = []

    for key, value in predictions.items():
        predicted_probs.append(value[0])
        true_labels.append(value[2])
        predicted_labels.append(value[1])

    logloss = log_loss(true_labels, predicted_probs)
    auc = roc_auc_score(true_labels, predicted_probs)
    f1 = f1_score(true_labels, predicted_labels)
    
    return logloss, auc, f1

def Evaluate(model, vids_path_dict, dataset, save_path):
    
    imgs_path = vids_path_dict[dataset][0]
    count = 0
    labels_map = ["fake", "real"]
    predictions = {}
    probs = {}
    for img in imgs_path:

        originallabel = None
        if 'fake' in img.split('/')[-3]:
            originallabel = 0
        elif 'real' in img.split('/')[-3]:
            originallabel = 1

        count+=1
        print(count)
        image = img.replace('\\','/')
        img = Image.open(image)
        tfms = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(),
               transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
        img = tfms(img)
        with torch.no_grad():
            outputs = model(img.unsqueeze(0).to(device))

        predicted_prob = []
        predicted_label = None
        for idx in torch.topk(outputs[0], k=1).indices.tolist():
            prob = torch.softmax(outputs[0], 0)[idx].item()

        if labels_map[idx] == 'fake':
            predicted_prob = 1 - prob
            predictions[image] = [predicted_prob, 0, originallabel]
        else:
            predicted_prob = prob
            print(predicted_prob)
            predictions[image] = [predicted_prob, 1, originallabel]
        print('--------------')
        
    if 'neuraltextures' in image:
        originallabel = 'NeuralTextures'
    elif 'face2face' in image:
        originallabel = 'Face2Face'
    elif 'deepfakes' in image:
        originallabel = 'DeepFakes'
    elif 'faceswap' in image:
        originallabel = 'FaceSwap'
        
    with open(save_path + '.csv', 'w') as csvfile:
        fieldnames = ['ImagePath', 'ProbabilityScore', 'PredictedLabel', 'OriginalLabel']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, lineterminator='\n',)

        writer.writeheader()
        for i in range(len(predictions)):
            img = list(predictions)[i]
            writer.writerow({'ImagePath': list(predictions)[i], 'ProbabilityScore': predictions[img][0], 'PredictedLabel': predictions[img][1], 'OriginalLabel': predictions[img][2]})
    
    
    logloss, auc, f1 = get_scores(predictions)
    fieldnames = ['LogLoss','AUC','F1-Score']
    with open(save_path + '.csv', 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, lineterminator='\n',)
        writer.writeheader()
        writer.writerow({'LogLoss': logloss, 'AUC': auc, 'F1-Score': f1})
    torch.cuda.empty_cache()
    #     break

In [1]:
for i in range(4):
    no_evaluation_on = datasets[i]
#     print("NO EVALUATION DATASET IS: ", no_evaluation_on)
    for model_name in models:
        for dataset in datasets:
#             if no_evaluation_on == dataset:
            if 1 == 2:
                pass
            else:
                # Load model weights
                PATH = '../model_weights/supervised_models/' + model_name + '/' + augs[0] + '/' + no_evaluation_on + '/' + '/'
                for file in os.listdir(PATH):
                    if file.endswith(".pth"):
                        model_path = os.path.join(PATH, file)
                        model_path = model_path.replace("//", "/")
                        print(model_path)
                        model = timm.create_model(model_name, pretrained=True, num_classes=2).to(device)
                        model.load_state_dict(torch.load(model_path), strict=True)
                        model.to(device)
                        model.eval()
                        print("loaded " + model_name + " weights!")
                        
                        if dataset == 'FakeAVCeleb':
                            #Evaluation code
                            print('Evaluating ' + model_name + ' on ' + dataset + ' trained on ' + no_evaluation_on)
                            save_path = ('CrossEvaluationTestScores/Image_by_Image_Evaluation/' + model_name + '_trained_on_' + 
                                         no_evaluation_on + '_evaluated_on_' + dataset)
                            Evaluate(model, test_set_paths_dictionary, dataset, save_path)

                        elif dataset == 'DFDC':
                            print('Evaluating ' + model_name + ' on ' + dataset + ' trained on ' + no_evaluation_on)
                            save_path = ('CrossEvaluationTestScores/Image_by_Image_Evaluation/' + model_name + '_trained_on_' + 
                                         no_evaluation_on + '_evaluated_on_' + dataset)
                            Evaluate(model, test_set_paths_dictionary, dataset, save_path)
                            
                        elif dataset == 'CelebDFV2':
                            print('Evaluating ' + model_name + ' on ' + dataset + ' trained on ' + no_evaluation_on)
                            save_path = ('CrossEvaluationTestScores/Image_by_Image_Evaluation/' + model_name + '_trained_on_' + 
                                         no_evaluation_on + '_evaluated_on_' + dataset)
                            Evaluate(model, test_set_paths_dictionary, dataset, save_path)
                            
                        elif dataset == 'FaceForensics':
                            print('Evaluating ' + model_name + ' on ' + dataset + ' trained on ' + no_evaluation_on)
                            save_path = ('CrossEvaluationTestScores/Image_by_Image_Evaluation/' + model_name + '_trained_on_' + 
                                         no_evaluation_on + '_evaluated_on_' + dataset)
                            Evaluate(model, test_set_paths_dictionary, dataset, save_path)
                                
                        print('====================================================================================================')
    print('')
    print('')

In [2]:
for i in range(4):
    no_evaluation_on = datasets[i]
#     print("NO EVALUATION DATASET IS: ", no_evaluation_on)
    for model_name in models:
        for dataset in datasets:
#             if no_evaluation_on == dataset:
            if 1 == 2:
                pass
            else:
                # Load model weights
                PATH = '../model_weights/supervised_models/' + model_name + '/' + augs[1] + '/' + no_evaluation_on + '/' + '/'
                for file in os.listdir(PATH):
                    if file.endswith(".pth"):
                        model_path = os.path.join(PATH, file)
                        model_path = model_path.replace("//", "/")
                        print(model_path)
                        model = timm.create_model(model_name, pretrained=True, num_classes=2).to(device)
                        model.load_state_dict(torch.load(model_path), strict=True)
                        model.to(device)
                        model.eval()
                        print("loaded " + model_name + " weights!")
                        
                        if dataset == 'FakeAVCeleb':
                            #Evaluation code
                            print('Evaluating ' + model_name + ' on ' + dataset + ' trained on ' + no_evaluation_on)
                            save_path = ('CrossEvaluationTestScores/Image/' + model_name + '_trained_on_' + 
                                         no_evaluation_on + '_evaluated_on_' + dataset)
                            predictions = Evaluate(model, test_set_paths_dictionary, dataset, save_path)

                        elif dataset == 'DFDC':
                            print('Evaluating ' + model_name + ' on ' + dataset + ' trained on ' + no_evaluation_on)
                            save_path = ('CrossEvaluationTestScores/Image_by_Image_Evaluation/' + model_name + '_trained_on_' + 
                                         no_evaluation_on + '_evaluated_on_' + dataset)
                            Evaluate(model, test_set_paths_dictionary, dataset, save_path)
                            
                        elif dataset == 'CelebDFV2':
                            print('Evaluating ' + model_name + ' on ' + dataset + ' trained on ' + no_evaluation_on)
                            save_path = ('CrossEvaluationTestScores/Image_by_Image_Evaluation/' + model_name + '_trained_on_' + 
                                         no_evaluation_on + '_evaluated_on_' + dataset)
                            Evaluate(model, test_set_paths_dictionary, dataset, save_path)
                            
                        elif dataset == 'FaceForensics':
                            print('Evaluating ' + model_name + ' on ' + dataset + ' trained on ' + no_evaluation_on)
                            save_path = ('CrossEvaluationTestScores/Image_by_Image_Evaluation/' + model_name + '_trained_on_' + 
                                         no_evaluation_on + '_evaluated_on_' + dataset)
                            Evaluate(model, test_set_paths_dictionary, dataset, save_path)
                                
                        print('====================================================================================================')
            break
        break
    break
                        
#     print('')
#     print('')